In [29]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [30]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump, symb_perf_stats_vectorized

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results

verbose = False  # True prints more output
look_back_days = -250 * 6  # subset df iloc days

In [31]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [32]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_OHLCVA[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAIN', 'AAMC', 'ABST', 'ACAB', 'ACAD', 'ACAQ', 'ACBA', 'ACDI', 'ACEV',
       'ACNT',
       ...
       'WSC', 'WSTG', 'WTMA', 'WWAC', 'XEL', 'XFIN', 'XPEL', 'XPER', 'YNDX',
       'ZNH'],
      dtype='object', length=730)


In [33]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AADI', 'AAIN', 'AAMC', 'AAN', 'ABCL', 'ABCM', 'ABNB', 'ACA', 'ACAB',
       'ACAQ',
       ...
       'YMAB', 'YOU', 'ZGN', 'ZI', 'ZIM', 'ZIP', 'ZLAB', 'ZM', 'ZNTL', 'ZS'],
      dtype='object', length=1113)


In [34]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume)  # combine symbols with no volume and no close
print(f'symbols_drop with duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
print(f'symbols_drop with unique symbols: {len(symbols_drop)}')

symbols_drop with duplicate symbols: 1843
symbols_drop with unique symbols: 1348


In [35]:
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_a

A                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2016-12-01   44.080002   44.099998   42.919998   43.209999   41.179451   
2016-12-02   43.270000   44.090000   43.270000   44.029999   41.960926   
2016-12-05   44.209999   44.689999   44.209999   44.529999   42.437420   
2016-12-06   44.580002   44.900002   44.200001   44.840000   42.732857   
2016-12-07   44.560001   44.990002   44.110001   44.990002   42.875816   
...                ...         ...         ...         ...         ...   
2022-11-09  138.309998  139.419998  136.660004  137.000000  137.000000   
2022-11-10  142.869995  146.720001  142.529999  146.300003  146.300003   
2022-11-11  147.119995  149.740005  146.350006  148.309998  148.309998   
2022-11-14  149.009995  149.710007  146.300003  146.380005  146.380005   
2022-11-15  148.270004  149.460007  146.494995  148.000000  148.000000   

                              AA                                   ...  \
               Volume       Open       High        Low      Close  ...   
Date                                                               ...   
2016-12-01  2823100.0  28.840000  29.420000  28.600000  28.879999  ...   
2016-12-02  2153200.0  28.860001  29.625000  28.730000  29.040001  ...   
2016-12-05  2495000.0  29.030001  31.277000  28.889999  31.219999  ...   
2016-12-06  1136700.0  30.709999  31.190001  30.209999  31.150000  ...   
2016-12-07  1815200.0  31.299999  31.889999  30.764999  30.900000  ...   
...               ...        ...        ...        ...        ...  ...   
2022-11-09   971200.0  42.529999  42.744999  40.340000  40.680000  ...   
2022-11-10  1591900.0  42.810001  45.490002  42.810001  43.830002  ...   
2022-11-11  1227500.0  45.259998  50.759998  45.040001  47.660000  ...   
2022-11-14  1220300.0  46.549999  49.049999  46.400002  48.220001  ...   
2022-11-15  1245147.0  49.599998  49.990002  48.439999  49.209999  ...   

                 ZUMZ                                     ZWS             \
                  Low      Close  Adj Close     Volume   Open       High   
Date                                                                       
2016-12-01  24.250000  24.549999  24.549999   698200.0  21.17  21.299999   
2016-12-02  22.049999  24.700001  24.700001  1315200.0  20.90  20.950001   
2016-12-05  24.299999  25.000000  25.000000   644500.0  20.48  20.500000   
2016-12-06  24.700001  24.900000  24.900000   437000.0  20.02  20.129999   
2016-12-07  24.500000  24.950001  24.950001   445500.0  19.91  20.309999   
...               ...        ...        ...        ...    ...        ...   
2022-11-09  20.639999  20.780001  20.780001   239800.0  22.26  22.799999   
2022-11-10  21.780001  23.510000  23.510000   327600.0  22.92  23.910000   
2022-11-11  23.670000  24.280001  24.280001   205000.0  23.59  24.480000   
2022-11-14  23.670000  23.709999  23.709999   235900.0  24.07  24.230000   
2022-11-15  23.950001  24.520000  24.520000   281961.0  24.24  25.020000   

                                                        
                  Low      Close  Adj Close     Volume  
Date                                                    
2016-12-01  20.110001  20.990000  20.990000  4646900.0  
2016-12-02  20.170000  20.270000  20.270000  4153000.0  
2016-12-05  19.770000  20.049999  20.049999  2287000.0  
2016-12-06  19.850000  20.000000  20.000000   839400.0  
2016-12-07  19.860001  20.240000  20.240000   862100.0  
...               ...        ...        ...        ...  
2022-11-09  21.910000  21.980000  21.980000  1013200.0  
2022-11-10  22.690001  23.440001  23.440001  2877700.0  
2022-11-11  23.549999  24.200001  24.200001  1534600.0  
2022-11-14  23.770000  23.809999  23.809999  1036700.0  
2022-11-15  24.025000  24.690001  24.690001  1564982.0  

[1500 rows x 13824 columns]

In [36]:
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
df_c

,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABB,...,ZBRA,ZD,ZEN,ZEUS,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-01,41.179451,28.605278,44.209743,31.474321,161.826752,25.616564,32.801022,49.750000,13.469035,16.741417,...,76.839996,64.347824,20.830000,24.115704,33.310001,36.316727,13.952978,47.597813,24.549999,20.990000
2016-12-02,41.960926,28.763758,44.547955,31.281822,161.239105,25.712488,33.244057,49.650002,13.410217,16.757681,...,78.349998,65.069565,21.170000,25.299479,33.330002,36.163956,13.208055,47.539707,24.700001,20.270000
2016-12-05,42.437420,30.923019,44.180756,31.618700,161.608780,25.527657,33.486465,50.250000,13.469035,17.131699,...,80.940002,66.530434,21.059999,27.285473,34.279999,36.442547,12.873795,48.236835,25.000000,20.049999
2016-12-06,42.732857,30.853685,44.866852,31.522451,164.243301,25.724184,33.780945,50.450001,13.527854,17.107307,...,81.610001,68.652176,21.170000,26.845232,38.070000,37.673775,12.969297,48.624123,24.900000,20.000000
2016-12-07,42.875816,30.606062,47.012108,31.474321,167.531845,25.976864,35.093487,51.000000,13.880752,17.156090,...,82.919998,68.130432,21.150000,26.943060,38.119999,38.006302,12.950197,49.631065,24.950001,20.240000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-09,137.000000,40.680000,13.980000,74.540001,175.350006,134.869995,27.219999,99.739998,35.389999,28.950001,...,226.880005,81.309998,76.400002,28.059999,31.850000,49.709999,18.680000,133.169998,20.780001,21.980000
2022-11-10,146.300003,43.830002,14.930000,78.510002,183.529999,146.869995,28.910000,100.150002,37.400002,30.559999,...,253.419998,86.610001,76.699997,30.469999,35.450001,53.869999,19.459999,142.610001,23.510000,23.440001
2022-11-11,148.309998,47.660000,14.880000,78.019997,186.389999,149.699997,28.580000,100.220001,39.240002,31.580000,...,263.380005,88.000000,76.699997,30.150000,37.480000,55.389999,21.230000,148.550003,24.280001,24.200001


In [43]:
print(f'symbols with no volume: {len(symbols_no_volume)}')
print(f'symbols with no close: {len(symbols_no_close)}')
print(f'unique symbols dropped from df_a and df_c: {len(symbols_drop)}')
print(f'symbols in df (df_c) after dropped symbols: {len(df_c.columns)}')
print(f'rows in df (df_c) after dropped symbols: {len(df_c)}')

symbols with no volume: 730
symbols with no close: 1113
unique symbols dropped from df_a and df_c: 1348
symbols in df (df_c) after dropped symbols: 2304
rows in df (df_c) after dropped symbols: 1500


In [45]:
# _periods = [-15, -30, -60, -120, -240]

perf_rank_dict = {}
syms_perf_rank = []  # list of lists to store top 10 ranked symbols
_periods = [-15, -30, -60, -120, -240]
for _period in _periods:
  f_name = 'period' + str(_period)

  _df_c = df_c[_period::]
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(_df_c)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']

  # write symbols' performance stats to dataframe
  df_ps = pd.DataFrame(caches_perf_stats_vect, columns=column_names)
  df_ps['r_CAGR/UI'] = df_ps['CAGR/UI'].rank(ascending=False)
  df_ps['r_CAGR/Std'] = df_ps['CAGR/Std'].rank(ascending=False)
  df_ps['r_Std/UI'] = df_ps['Std/UI'].rank(ascending=False)
  
  _dict = {}
  cols_sort = ['r_CAGR/UI', 'r_CAGR/Std', 'r_Std/UI']
  print(f'{f_name} top 100 symbols')  
  for col in cols_sort:
    symbols_top_100 = df_ps.sort_values(by=[col]).head(100).symbol.values
    syms_perf_rank.append(list(symbols_top_100))
    print(f'{col}: {symbols_top_100}')
    _dict[col] = symbols_top_100
    perf_rank_dict[f'{f_name}'] = _dict
  print(' ')

pickle_dump(perf_rank_dict, path_data_dump, filename_pickled_perf_rank_dict)
print(f'Pickled perf_rank_dict to: {path_data_dump}{filename_pickled_perf_rank_dict}\n')
print(f'perf_rank_dict:\n{perf_rank_dict}\n')
print(f'syms_perf_rank: {syms_perf_rank}')


period-15 top 100 symbols
r_CAGR/UI: ['NRIM' 'VCYT' 'PKOH' 'LOPE' 'AXON' 'AMKR' 'DMRC' 'RMBS' 'MLAB' 'GBX'
 'PKX' 'RCKY' 'LGND' 'ABMD' 'WNC' 'BA' 'RUN' 'AIMC' 'WYNN' 'ETD' 'LSCC'
 'AE' 'CIR' 'TSM' 'AAON' 'HEES' 'THRM' 'TSE' 'MTSI' 'XPO' 'NEOG' 'ACLS'
 'DD' 'SRI' 'ATNI' 'HOFT' 'HLI' 'SMG' 'DAN' 'GPRE' 'IVZ' 'SHOP' 'TSBK'
 'AU' 'SMCI' 'CHGG' 'PIPR' 'KLAC' 'SEIC' 'OSPN' 'LVS' 'NOMD' 'AEIS' 'SAIA'
 'BVH' 'YUMC' 'MOD' 'FICO' 'NYT' 'UFPT' 'NVDA' 'PZZA' 'CZR' 'TEX' 'MAC'
 'SKX' 'TPL' 'OII' 'NPO' 'RNR' 'CNK' 'NBHC' 'TEN' 'JD' 'GS' 'PRDO' 'LRCX'
 'AVNS' 'AMAT' 'SPWR' 'APTV' 'MYE' 'GVA' 'MCY' 'AMG' 'SONY' 'PDFS' 'PLNT'
 'PTMN' 'PRG' 'AMD' 'AXSM' 'HVT' 'GE' 'FOXF' 'DENN' 'PUK' 'QNST' 'GDO' 'E']
r_CAGR/Std: ['VCYT' 'DMRC' 'PKOH' 'AMKR' 'MLAB' 'LGND' 'CIR' 'GBX' 'RUN' 'AXON' 'WYNN'
 'RCKY' 'LSCC' 'RMBS' 'WNC' 'PKX' 'ETD' 'BA' 'TSE' 'SRI' 'ACLS' 'AIMC'
 'THRM' 'XPO' 'SHOP' 'TSM' 'HEES' 'SMCI' 'AE' 'SMG' 'NEOG' 'FICO' 'MTSI'
 'JD' 'AAON' 'HOFT' 'SAIA' 'ATNI' 'ABMD' 'AU' 'AXSM' 'GPRE' 'DAN' 'CHGG'
 'O

In [38]:
# _periods = [-15, -30, -60, -120, -240]

perf_rank_dict = {}
syms_perf_rank = []  # list of lists to store top 10 ranked symbols
_periods = [-15, -30, -60, -120, -240]
for _period in _periods:
  f_name = 'period' + str(_period)

  _df_c = df_c[_period::]
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(_df_c)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']

  # write symbols' performance stats to dataframe
  df_ps = pd.DataFrame(caches_perf_stats_vect, columns=column_names)
  df_ps['r_CAGR/UI'] = df_ps['CAGR/UI'].rank(ascending=False)
  df_ps['r_CAGR/Std'] = df_ps['CAGR/Std'].rank(ascending=False)
  df_ps['r_Std/UI'] = df_ps['Std/UI'].rank(ascending=False)
  
  _dict = {}
  cols_sort = ['r_CAGR/UI', 'r_CAGR/Std', 'r_Std/UI']
  print(f'{f_name} top 100 symbols')  
  for col in cols_sort:
    symbols_top_100 = df_ps.sort_values(by=[col]).head(100).symbol.values
    syms_perf_rank.append(list(symbols_top_100))
    print(f'{col}: {symbols_top_100}')
    _dict[col] = symbols_top_100
    perf_rank_dict[f'{f_name}'] = _dict
  print(' ')

pickle_dump(perf_rank_dict, path_data_dump, filename_pickled_perf_rank_dict)
print(f'Pickled perf_rank_dict to: {path_data_dump}{filename_pickled_perf_rank_dict}\n')
print(f'perf_rank_dict:\n{perf_rank_dict}\n')
print(f'syms_perf_rank: {syms_perf_rank}')


period-15 top 100 symbols
r_CAGR/UI: ['NRIM' 'VCYT' 'PKOH' 'LOPE' 'AXON' 'AMKR' 'DMRC' 'RMBS' 'MLAB' 'GBX'
 'PKX' 'RCKY' 'LGND' 'ABMD' 'WNC' 'BA' 'RUN' 'AIMC' 'WYNN' 'ETD' 'LSCC'
 'AE' 'CIR' 'TSM' 'AAON' 'HEES' 'THRM' 'TSE' 'MTSI' 'XPO' 'NEOG' 'ACLS'
 'DD' 'SRI' 'ATNI' 'HOFT' 'HLI' 'SMG' 'DAN' 'GPRE' 'IVZ' 'SHOP' 'TSBK'
 'AU' 'SMCI' 'CHGG' 'PIPR' 'KLAC' 'SEIC' 'OSPN' 'LVS' 'NOMD' 'AEIS' 'SAIA'
 'BVH' 'YUMC' 'MOD' 'FICO' 'NYT' 'UFPT' 'NVDA' 'PZZA' 'CZR' 'TEX' 'MAC'
 'SKX' 'TPL' 'OII' 'NPO' 'RNR' 'CNK' 'NBHC' 'TEN' 'JD' 'GS' 'PRDO' 'LRCX'
 'AVNS' 'AMAT' 'SPWR' 'APTV' 'MYE' 'GVA' 'MCY' 'AMG' 'SONY' 'PDFS' 'PLNT'
 'PTMN' 'PRG' 'AMD' 'AXSM' 'HVT' 'GE' 'FOXF' 'DENN' 'PUK' 'QNST' 'GDO' 'E']
r_CAGR/Std: ['VCYT' 'DMRC' 'PKOH' 'AMKR' 'MLAB' 'LGND' 'CIR' 'GBX' 'RUN' 'AXON' 'WYNN'
 'RCKY' 'LSCC' 'RMBS' 'WNC' 'PKX' 'ETD' 'BA' 'TSE' 'SRI' 'ACLS' 'AIMC'
 'THRM' 'XPO' 'SHOP' 'TSM' 'HEES' 'SMCI' 'AE' 'SMG' 'NEOG' 'FICO' 'MTSI'
 'JD' 'AAON' 'HOFT' 'SAIA' 'ATNI' 'ABMD' 'AU' 'AXSM' 'GPRE' 'DAN' 'CHGG'
 'O

In [44]:
syms_perf_rank  # list of lists
l_syms_perf_rank = [val for sublist in syms_perf_rank for val in sublist]  # flatten list of lists

from collections import Counter
c = Counter(l_syms_perf_rank)  # count symbols and count
print(c) 
c_tuples = c.most_common()  # e.g [('AKRO', 6), ('IMVT', 4), ... ('ADEA', 3)]
r_all_ranks = [symbol for symbol, count in c_tuples]  # select just the symbols without the frequency counts
r_all_ranks  # list of most common symbols in syms_perf_rank in descending order
pickle_dump(r_all_ranks, path_data_dump, filename_pickled_r_all_ranks)
r_all_ranks

Counter({'TPL': 14, 'NRIM': 11, 'TSBK': 11, 'AVEO': 11, 'ASC': 11, 'INSW': 11, 'LOPE': 10, 'AXON': 10, 'AAON': 10, 'MOD': 10, 'UFPT': 10, 'SANM': 10, 'RMBS': 9, 'AE': 9, 'RELL': 9, 'ELF': 9, 'ABMD': 8, 'WNC': 8, 'AIMC': 8, 'HLI': 8, 'CHGG': 8, 'GILD': 8, 'OMAB': 8, 'CHUY': 8, 'EME': 8, 'LANC': 8, 'HFWA': 8, 'STNG': 8, 'NPO': 7, 'IBA': 7, 'EURN': 7, 'SLB': 7, 'FTI': 7, 'FCBC': 7, 'GPC': 7, 'ANIK': 7, 'ADEA': 7, 'TNK': 7, 'AMKR': 6, 'DMRC': 6, 'THRM': 6, 'SRI': 6, 'MAC': 6, 'OII': 6, 'TEN': 6, 'MYE': 6, 'DENN': 6, 'OSBC': 6, 'RETA': 6, 'PLOW': 6, 'TR': 6, 'ODP': 6, 'STBA': 6, 'TBBK': 6, 'MEDP': 6, 'PRTA': 6, 'NTNX': 6, 'MYOV': 6, 'MODN': 6, 'ERIE': 6, 'NFLX': 6, 'DXCM': 6, 'AGYS': 6, 'TWNK': 6, 'TRQ': 6, 'VCYT': 5, 'GBX': 5, 'PKX': 5, 'BA': 5, 'ETD': 5, 'CIR': 5, 'ATNI': 5, 'DAN': 5, 'SMCI': 5, 'PIPR': 5, 'CZR': 5, 'PRDO': 5, 'AXSM': 5, 'CRAI': 5, 'AEHR': 5, 'RCL': 5, 'HALO': 5, 'TDW': 5, 'BBSI': 5, 'NBTB': 5, 'BIIB': 5, 'TTC': 5, 'HTBK': 5, 'EVI': 5, 'FSLR': 5, 'TCBK': 5, 'CCRN': 5, 'CH

['TPL',
 'NRIM',
 'TSBK',
 'AVEO',
 'ASC',
 'INSW',
 'LOPE',
 'AXON',
 'AAON',
 'MOD',
 'UFPT',
 'SANM',
 'RMBS',
 'AE',
 'RELL',
 'ELF',
 'ABMD',
 'WNC',
 'AIMC',
 'HLI',
 'CHGG',
 'GILD',
 'OMAB',
 'CHUY',
 'EME',
 'LANC',
 'HFWA',
 'STNG',
 'NPO',
 'IBA',
 'EURN',
 'SLB',
 'FTI',
 'FCBC',
 'GPC',
 'ANIK',
 'ADEA',
 'TNK',
 'AMKR',
 'DMRC',
 'THRM',
 'SRI',
 'MAC',
 'OII',
 'TEN',
 'MYE',
 'DENN',
 'OSBC',
 'RETA',
 'PLOW',
 'TR',
 'ODP',
 'STBA',
 'TBBK',
 'MEDP',
 'PRTA',
 'NTNX',
 'MYOV',
 'MODN',
 'ERIE',
 'NFLX',
 'DXCM',
 'AGYS',
 'TWNK',
 'TRQ',
 'VCYT',
 'GBX',
 'PKX',
 'BA',
 'ETD',
 'CIR',
 'ATNI',
 'DAN',
 'SMCI',
 'PIPR',
 'CZR',
 'PRDO',
 'AXSM',
 'CRAI',
 'AEHR',
 'RCL',
 'HALO',
 'TDW',
 'BBSI',
 'NBTB',
 'BIIB',
 'TTC',
 'HTBK',
 'EVI',
 'FSLR',
 'TCBK',
 'CCRN',
 'CHCO',
 'WING',
 'VRTX',
 'NBIX',
 'MGI',
 'DGII',
 'CPRX',
 'CCBG',
 'TNP',
 'HEES',
 'XPO',
 'DD',
 'HOFT',
 'SMG',
 'OSPN',
 'FICO',
 'TEX',
 'RNR',
 'GE',
 'E',
 'WLFC',
 'MGRC',
 'RFP',
 'CAT',
 'UNFI'

In [40]:
l_LNG = ['LNG', 'CHK', 'GLNG']
for sym in l_LNG:
  print(sym)
  print(df[sym].tail())

LNG
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-11-09  172.860001  172.990005  161.729996  162.869995  162.869995   
2022-11-10  165.929993  168.869995  164.440002  167.320007  167.320007   
2022-11-11  167.800003  168.710007  158.630005  161.759995  161.759995   
2022-11-14  161.000000  166.639999  160.830002  164.500000  164.500000   
2022-11-15  164.389999  165.699997  160.880005  165.369995  165.369995   

               Volume  
Date                   
2022-11-09  3572100.0  
2022-11-10  2468600.0  
2022-11-11  4437700.0  
2022-11-14  2625900.0  
2022-11-15  2516550.0  
CHK
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-11-09  102.260002  102.730003   98.620003   98.709999   95.664764   
2022-11-10  101.220001  102.239998   99.269997  101.809998   98.669121   
2022-11-1